## Submission. Расчет на финальной тестовой

### Libraries

In [ ]:
#!pip install catboost -q
#!pip install shap -q
#!pip install pyarrow -q
#!pip install fastparquet -q

In [1]:
import json
import ast
from functools import partial
from typing import List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from catboost import CatBoostClassifier, Pool
from catboost.utils import eval_metric
from scipy.spatial.distance import cosine, euclidean
from sklearn.metrics import pairwise_distances, roc_curve, precision_recall_curve
from sklearn.metrics import  auc, accuracy_score as acc, roc_auc_score, f1_score, precision_score
from sklearn.model_selection import train_test_split,RandomizedSearchCV,GridSearchCV
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OrdinalEncoder

from nltk.metrics.distance import edit_distance

import joblib

from tqdm import tqdm
import warnings

# настройки
tqdm.pandas()
warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

# константы
RANDOM_STATE = 42

### Model loading

In [2]:
model = CatBoostClassifier()

In [3]:
model.load_model('final_Ozon.cbm')

### Functions

In [4]:
def new_col(train_data):


    # переводим текст категорий в словарь
    train_data['attributes_set'] = train_data.characteristic_attributes_mapping.fillna('{}').apply(ast.literal_eval)
    
    train_data['cat2'] = train_data['categories'].apply(lambda x: json.loads(x).get('2'))
    train_data['cat3'] = train_data['categories'].apply(lambda x: json.loads(x).get('3'))  
    train_data['cat4'] = train_data['categories'].apply(lambda x: json.loads(x).get('4'))  
    
    # Переведем двумрные эмбеддинги в одномерные
    train_data['main_pic_embeddings_resnet_v1_new'] = train_data['main_pic_embeddings_resnet_v1'].apply(lambda x: x[0])
    
    return train_data

In [5]:
def cat3_grouped(train_data, FOR_REST):
    cat3_counts = train_data["cat3"].value_counts().to_dict()

    cntr = 0
    for cat3 in cat3_counts:
        if cat3_counts[cat3] < FOR_REST:
            cntr += cat3_counts[cat3]   
    print(cntr)

    train_data["cat3_grouped"] = train_data["cat3"].apply(lambda x: x if cat3_counts[x] > FOR_REST else "rest")

    return train_data

In [6]:
# выделим самые популярные цвета и поместим все в матрицу цветов
def color(df):
       
    df['black'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('черный' in x) or ('black') in x else 0)
    df['silver'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('silver'in x) or ('серебристый' in x) or ('серебряный' in x) else 0)
    df['rose'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('pink' in x) or ('розовый'  in x) else 0)
    df['red'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('red' in x) or ('красный' in x) else 0)
    df['white'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('white' in x) or ('белый' in x) else 0)
    df['blue'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('blue' in x) or ('синий' in x) or ('голубой' in x)  else 0)
    df['multicol'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('разноцветный' in x) or ('цветной' in x) or ('многоцветный' in x) else 0)
    df['green'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('green' in x) or ('зеленый' in x) else 0)
    df['yellow'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('yellow' in x) or ('желтый' in x) else 0)
    df['gold'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('gold' in x) or ('золотой' in x) else 0)
    df['purple'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('purple' in x) or ('пурпурный' in x)  or ('сиреневый' in x) or ('фиолетовый' in x) else 0)
    df['orange'] = df['color_parsed'].astype(str).apply(lambda x: 1 if 'оранжевый' in x else 0)
    df['brown'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('коричневый' in x) or ('brown' in x) or ('темно-коричневый' in x) else 0)
    df['grey'] = df['color_parsed'].astype(str).apply(lambda x: 1 if ('серый' in x) or ('grey' in x) or ('темно-серый' in x) else 0)
    
    name_color = ['black', 'silver', 'rose', 'red', 'white', 'blue', 'multicol', 
                  'green', 'yellow', 'gold', 'purple', 'orange', 'brown', 'grey'] 
    
    df['clr_vect'] = df[name_color].values.tolist()
    df = df.drop(name_color, axis=1)
    df = df.drop(['color_parsed'], axis=1)
     
    return df

In [7]:
# объединим данные по парам товаров
def merge_f(pairs, data): 
    features = pairs.merge(data.add_suffix('1'), on='variantid1').merge(data.add_suffix('2'), on='variantid2')
    features.rename(columns={'cat31': 'cat3', 'cat3_grouped1': 'cat3_grouped'}, inplace=True)
    return features

In [8]:
# добавим признаки взаимосвязи характеристик
def is_number(value):
    try:
        float(value)
        return True
    except ValueError:
        return False

def compare_attr(set1, set2):
    keys1 = set(set1.keys())
    keys2 = set(set2.keys())
    
    # Отношение общих ключей к сумме ключей
    ratio = len(keys2 & keys1) / (len(keys2) + len(keys1))
    
    # Количество одинаковых значений в общих ключах
    d = 0
    for i in keys2 & keys1:
        if set1[i][0] == set2[i][0]:
            d += 1
    
    # Количество общих ключей
    common_keys = len(keys1 & keys2)
    
    # Количество ключей с одинаковыми значениями
    same_value_keys = sum(1 for key in keys1 & keys2 if set1[key] == set2[key])
    
    # Среднее значение числовых характеристик для первого товара
    mean_value1 = np.mean([float(value[0]) for key, value in set1.items() if is_number(value[0])])
    
    # Среднее значение числовых характеристик для второго товара
    mean_value2 = np.mean([float(value[0]) for key, value in set2.items() if is_number(value[0])])
    
    return ratio, d, common_keys, same_value_keys, mean_value1, mean_value2

In [9]:
#Похожесть атрибутов
def calculate_statistical_features(embedding1, embedding2):
    
    # Вычисляем разность между двумя эмбеддингами
    diff = embedding1 - embedding2
    
    # Вычисляем среднее значение разности
    mean = np.mean(diff)
    
    # Вычисляем медиану разности
    median = np.median(diff)
    
    # Вычисляем стандартное отклонение разности
    std = np.std(diff)
    
    # Возвращаем вычисленные значения
    return mean, median, std

In [10]:
# добавим коэффициент Жаккара
def calculate_jaccard_similarity(text1, text2):
    # Заменяем значения None на пустые строки
    text1 = '' if text1 is None else text1
    text2 = '' if text2 is None else text2
    
    # Заменяем значения типа float на пустые строки
    text1 = '' if isinstance(text1, float) else text1
    text2 = '' if isinstance(text2, float) else text2
    
    # Инициализируем векторизатор с бинарными значениями
    vectorizer = CountVectorizer(binary=True)

    try:
        # Преобразуем тексты в векторы
        X = vectorizer.fit_transform([text1, text2])
    except ValueError:
        # Если после предобработки текстов не осталось слов для анализа,
        # возвращаем значение 0
        return 0
    
    # Преобразуем тексты в векторы
    X = vectorizer.fit_transform([text1, text2])
    
    # Вычисляем пересечение между векторами
    intersection = X[0].multiply(X[1]).sum()
    
    # Вычисляем объединение между векторами
    union = X.sum() - intersection
    
    # Возвращаем коэффициент Жаккара
    return intersection / union

In [11]:
def get_pic_features(main_pic_embeddings_1,
                     main_pic_embeddings_2,
                     percentiles: List[int]):
    """
    Calculate distances percentiles for 
    pairwise pic distances. Percentiles are useful 
    when product has several pictures.
    """
    
    if main_pic_embeddings_1 is not None and main_pic_embeddings_2 is not None:
        main_pic_embeddings_1 = np.array([x for x in main_pic_embeddings_1])
        main_pic_embeddings_2 = np.array([x for x in main_pic_embeddings_2])
        
        dist_m = pairwise_distances(
            main_pic_embeddings_1, main_pic_embeddings_2
        )
    else:
        dist_m = np.array([[-1]])

    pair_features = []
    pair_features += np.percentile(dist_m, percentiles).tolist()

    return pair_features


def text_dense_distances(ozon_embedding, comp_embedding):
    """
    Calculate Euclidean and Cosine distances between
    ozon_embedding and comp_embedding.
    """
    pair_features = []
    if ozon_embedding is None or comp_embedding is None:
        pair_features = [-1, -1]
    
    elif len(ozon_embedding) == 0 or len(comp_embedding) == 0:
        pair_features = [-1, -1]
    
    else:
        pair_features.append(
            euclidean(ozon_embedding, comp_embedding)
        )
        cosine_value = cosine(ozon_embedding, comp_embedding)
        
        pair_features.append(cosine_value)

    return pair_features

In [12]:
def manhattan_distance(emb1, emb2):
    """
    Функция для расчета манхэттенского расстояния между двумя одномерными эмбеддингами в строке датафрейма
    :return: манхэттенское расстояние между эмбеддингами в строке
    """
    return np.abs(emb1 - emb2)

In [13]:
def calculate_statistics(arr):
    """
    Функция для расчета статистических мер для массива
    :param arr: массив значений
    :return: кортеж из среднего значения, медианы и стандартного отклонения
    """
    mean = np.mean(arr)
    median = np.median(arr)
    std = np.std(arr)
    
    return mean, median, std

In [14]:
def levenshtein_distance(text1, text2):
    """
    Функция для расчета расстояния Левенштейна между двумя текстами
    :param text1: первый текст
    :param text2: второй текст
    :return: расстояние Левенштейна между text1 и text2
    """
    return edit_distance(text1, text2)

In [15]:
# drop
def drop_f(features):
    features = features.drop([
        'name_bert_641', 
        'name_bert_642', 
        'main_pic_embeddings_resnet_v11', 
        'main_pic_embeddings_resnet_v12',
        'main_pic_embeddings_resnet_v1_new1', 
        'main_pic_embeddings_resnet_v1_new2',
        'name_bert7681', 
        'name_bert7682',
        'categories1',
        'categories2',
        'pic_embeddings_resnet_v11',
        'pic_embeddings_resnet_v12',       
        'cat3_grouped2',
        'attributes_set1', 
        'attributes_set2', 
        'clr_vect1', 
        'clr_vect2'], axis=1)  
    
    return features

### Data loading

In [16]:
test_data = pd.read_parquet('test_data.parquet', engine='auto')
test_pairs_wo_target = pd.read_parquet('test_pairs_wo_target.parquet', engine='auto')

In [17]:
%%time
# грузим дополнительные ембеддинги названий
name_bert = pd.read_parquet('name_bert768_a_test.parquet')

CPU times: total: 531 ms
Wall time: 511 ms


In [18]:
# объединим датасеты с новыми эмбеддингами
test_data = test_data.merge(name_bert, on='variantid')

display(test_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35730 entries, 0 to 35729
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   variantid                          35730 non-null  int64 
 1   name                               35730 non-null  object
 2   categories                         35730 non-null  object
 3   color_parsed                       26089 non-null  object
 4   pic_embeddings_resnet_v1           19977 non-null  object
 5   main_pic_embeddings_resnet_v1      35730 non-null  object
 6   name_bert_64                       35730 non-null  object
 7   characteristic_attributes_mapping  35726 non-null  object
 8   name_bert768                       35730 non-null  object
dtypes: int64(1), object(8)
memory usage: 2.7+ MB


None

In [19]:
# удалим дубликаты по колонке variantid
test_data = test_data.drop_duplicates('variantid')

display(test_data.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35730 entries, 0 to 35729
Data columns (total 9 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   variantid                          35730 non-null  int64 
 1   name                               35730 non-null  object
 2   categories                         35730 non-null  object
 3   color_parsed                       26089 non-null  object
 4   pic_embeddings_resnet_v1           19977 non-null  object
 5   main_pic_embeddings_resnet_v1      35730 non-null  object
 6   name_bert_64                       35730 non-null  object
 7   characteristic_attributes_mapping  35726 non-null  object
 8   name_bert768                       35730 non-null  object
dtypes: int64(1), object(8)
memory usage: 2.7+ MB


None

### Encoder

In [20]:
encoder = joblib.load('encoder_ozon.pkl')
# закодируем колонки cat3 и cat4
cat_col = ['cat3', 'cat4'] #список колонок для кодирования

### *Предобработка тестовой выборки*

In [21]:
%%time

REST_test = 50

test_data = new_col(test_data)
test_data = cat3_grouped(test_data, REST_test)
test_data[cat_col] = encoder.transform(test_data[cat_col]) 
test_data = color(test_data)

# Объединение таблиц
features_test = merge_f(test_pairs_wo_target, test_data)


features_test[['attr_dist', 
               'attr_acc', 
               'common_keys', 
               'same_value_keys', 
               'mean_value1', 
               'mean_value2']] = features_test[['attributes_set1', 'attributes_set2']].progress_apply(
    lambda x: pd.Series(compare_attr(*x)), axis=1
)

features_test['mean_value1'] = features_test['mean_value1'].fillna(0)
features_test['mean_value2'] = features_test['mean_value2'].fillna(0)

449


100%|██████████| 18084/18084 [00:05<00:00, 3486.04it/s]

CPU times: total: 18.5 s
Wall time: 18.5 s


In [22]:
features_test['main_pic_embeddings_resnet_v1_new1'] = features_test['main_pic_embeddings_resnet_v11'].apply(lambda x: x[0])
features_test['main_pic_embeddings_resnet_v1_new2'] = features_test['main_pic_embeddings_resnet_v12'].apply(lambda x: x[0])

In [23]:
%%time
# Добавляем новые столбцы в датафрейм features_test с вычисленными статистическими характеристиками разности 
# между эмбеддингами названий товаров
features_test[['name_mean_diff', 'name_median_diff', 'name_std_diff']] = features_test.progress_apply(
    lambda x: calculate_statistical_features(
        x['name_bert7681'], x['name_bert7682']), axis=1, result_type='expand'
)

# Добавляем новые столбцы в датафрейм features_test с вычисленными статистическими характеристиками разности 
# между эмбеддингами названий товаров от ozon
features_test[['ozon_name_mean_diff', 'ozon_name_median_diff', 'ozon_name_std_diff']] = features_test.progress_apply(
    lambda x: calculate_statistical_features(
        x['name_bert_641'], x['name_bert_642']), axis=1, result_type='expand'
)

# Добавляем новые столбцы в датафрейм features_test с вычисленными статистическими характеристиками разности 
# между эмбеддингами картинок товаров
features_test[['main_pic_mean_diff', 'main_pic_median_diff', 'main_pic_std_diff']] = features_test.progress_apply(
    lambda x: calculate_statistical_features(
        x['main_pic_embeddings_resnet_v1_new1'], x['main_pic_embeddings_resnet_v1_new2']), axis=1, result_type='expand'
)

100%|██████████| 18084/18084 [00:02<00:00, 7077.84it/s]

CPU times: total: 8.77 s
Wall time: 8.72 s


In [24]:
%%time
# Добавляем новый столбец в датафрейм features_test с вычисленным коэффициентом Жаккара между названиями товаров
features_test['name_jaccard_similarity'] = features_test.progress_apply(
    lambda x: calculate_jaccard_similarity(x['name1'], x['name2']), axis=1
)

# Добавляем новый столбец в датафрейм features_test с вычисленным коэффициентом Жаккара между атрибутами товаров
features_test['attributes_jaccard_similarity'] = features_test.progress_apply(
    lambda x: calculate_jaccard_similarity(x['characteristic_attributes_mapping1'], x['characteristic_attributes_mapping2']), axis=1
)

100%|██████████| 18084/18084 [00:35<00:00, 515.36it/s]

CPU times: total: 1min 5s
Wall time: 1min 4s


In [25]:
%%time
get_pic_features_func = partial(get_pic_features, percentiles=[0, 25, 50])

features_test[["pic_dist_0_perc", "pic_dist_25_perc", "pic_dist_50_perc"]] = (
    features_test[["pic_embeddings_resnet_v11", "pic_embeddings_resnet_v12"]].progress_apply(
        lambda x: pd.Series(get_pic_features_func(*x)), axis=1))

features_test[["main_pic_dist_0_perc", "main_pic_dist_25_perc", "main_pic_dist_50_perc"]] = (
    features_test[["main_pic_embeddings_resnet_v11", "main_pic_embeddings_resnet_v12"]].progress_apply(
        lambda x: pd.Series(get_pic_features_func(*x)), axis=1))

features_test[["euclidean_main_pic_dist", "cosine_main_pic_dist"]] = (
    features_test[["main_pic_embeddings_resnet_v1_new1", "main_pic_embeddings_resnet_v1_new2"]].progress_apply(
        lambda x: pd.Series(text_dense_distances(*x)), axis=1))

features_test[["euclidean_color_dist", "cosine_color_dist"]] = (
    features_test[["clr_vect1", "clr_vect2"]].progress_apply(
        lambda x: pd.Series(text_dense_distances(*x)), axis=1))

features_test[["euclidean_name_bert_dist", "cosine_name_bert_dist"]] = (
    features_test[["name_bert_641", "name_bert_642"]].progress_apply(
        lambda x: pd.Series(text_dense_distances(*x)), axis=1))

features_test[["euclidean_name_embedding_dist", "cosine_name_embedding_dist"]] = (
    features_test[["name_bert7681", "name_bert7682"]].progress_apply(
        lambda x: pd.Series(text_dense_distances(*x)), axis=1))


100%|██████████| 18084/18084 [00:06<00:00, 2923.02it/s]

CPU times: total: 42.6 s
Wall time: 42.3 s


In [26]:
# Расчет features_test манхэттенского расстояния между признаками name_bert_641 и name_bert_642
features_test['ozon_bert_manhattan_distance'] = features_test.progress_apply(
    lambda x: manhattan_distance(x['name_bert_641'], x['name_bert_642']), axis=1
)

# Расчет features_test манхэттенского расстояния между признаками main_pic_embeddings_resnet_v1_new1 и main_pic_embeddings_resnet_v1_new2
features_test['main_pic_manhattan_distance'] = features_test.progress_apply(
    lambda x: manhattan_distance(x['main_pic_embeddings_resnet_v1_new1'], x['main_pic_embeddings_resnet_v1_new2']), axis=1
)

# Расчет features_test манхэттенского расстояния между признаками name_embedding1 и name_embedding2
features_test['name_emb_manhattan_distance'] = features_test.progress_apply(
    lambda x: manhattan_distance(x['name_bert7681'], x['name_bert7682']), axis=1
)

100%|██████████| 18084/18084 [00:00<00:00, 22670.48it/s]


In [27]:
%%time
# расчитываем среднее, медиану и стд для ozon_bert_manhattan_distance
features_test[["mean_ozon_bert_manhattan_distance", 
          "median_ozon_bert_manhattan_distance", 
          "std_ozon_bert_manhattan_distance"]] = (features_test['ozon_bert_manhattan_distance'].progress_apply(
        lambda x: pd.Series(calculate_statistics(x))))

# расчитываем среднее, медиану и стд для main_pic_manhattan_distance
features_test[["mean_main_pic_manhattan_distance", 
          "median_main_pic_manhattan_distance", 
          "std_main_pic_manhattan_distance"]] = (features_test['main_pic_manhattan_distance'].progress_apply(
        lambda x: pd.Series(calculate_statistics(x))))

# расчитываем среднее, медиану и стд для name_emb_manhattan_distance
features_test[["mean_name_emb_manhattan_distance", 
          "median_name_emb_manhattan_distance", 
          "std_name_emb_manhattan_distance"]] = (features_test['name_emb_manhattan_distance'].progress_apply(
        lambda x: pd.Series(calculate_statistics(x))))

100%|██████████| 18084/18084 [00:05<00:00, 3189.98it/s]

CPU times: total: 14.7 s
Wall time: 14.7 s


In [28]:
%%time
# расчитаем расстояния Левенштейна для name1 и name2
features_test[["name_levenshtein_distance"]] = (
    features_test[["name1", "name2"]].progress_apply(
        lambda x: pd.Series(levenshtein_distance(*x)), axis=1))

100%|██████████| 18084/18084 [02:37<00:00, 115.02it/s]

CPU times: total: 2min 36s
Wall time: 2min 37s


In [29]:
# заполним пропуски в characteristic_attributes_mapping1 и characteristic_attributes_mapping2
features_test = drop_f(features_test)
features_test[['characteristic_attributes_mapping1', 
               'characteristic_attributes_mapping2']] = features_test[['characteristic_attributes_mapping1', 
                                                                       'characteristic_attributes_mapping2']].fillna('{}')

In [30]:
display(features_test.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18084 entries, 0 to 18083
Data columns (total 58 columns):
 #   Column                               Non-Null Count  Dtype  
---  ------                               --------------  -----  
 0   variantid1                           18084 non-null  int64  
 1   variantid2                           18084 non-null  int64  
 2   cat3_grouped                         18084 non-null  object 
 3   name1                                18084 non-null  object 
 4   characteristic_attributes_mapping1   18084 non-null  object 
 5   cat21                                18084 non-null  object 
 6   cat3                                 18084 non-null  float64
 7   cat41                                18084 non-null  float64
 8   cat3_grouped                         18084 non-null  object 
 9   name2                                18084 non-null  object 
 10  characteristic_attributes_mapping2   18084 non-null  object 
 11  cat22                       

None

In [31]:
feats = joblib.load('feats_ozon.joblib') #загружаем список колонок, необходимых для предсказаний

In [32]:
submission_example = features_test.copy()

submission_example["target"] = model.predict_proba(features_test[feats])[:, 1]
submission_example = submission_example[["variantid1", "variantid2", "target"]]
submission_example.head(3)

,variantid1,variantid2,target
0,52076340,290590137,0.148339
1,64525522,204128919,0.504336
2,77243372,479860557,0.568597


In [33]:
s = submission_example.drop_duplicates().merge(
      features_test[["variantid1", "variantid2"]].drop_duplicates(["variantid1", "variantid2"]),
      on=["variantid1", "variantid2"]
)

s.head(3)

,variantid1,variantid2,target
0,52076340,290590137,0.148339
1,64525522,204128919,0.504336
2,77243372,479860557,0.568597


In [34]:
features_test.duplicated(["variantid1", "variantid2"]).sum()

0

In [35]:
s.target.min()

0.004518014998942482

In [36]:
s.target.max()

0.9936960954617106

In [37]:
# s = s.drop_duplicates(["variantid1", "variantid2"])

In [40]:
s.to_csv("submission_Cat_29038.csv", index=False)

In [39]:
s.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18084 entries, 0 to 18083
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   variantid1  18084 non-null  int64  
 1   variantid2  18084 non-null  int64  
 2   target      18084 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 565.1 KB
